In [2]:
'''
1. Read a melville-moby_dick text file and print the first 15 lines.
2. How many words are in the book?
3. How many unique words are in the book?
4. What is the most common word in the book?
5. What is the most common word in the book which is not a stop-word?
'''

'\n1. Read a melville-moby_dick text file and print the first 15 lines.\n2. How many words are in the book?\n3. How many unique words are in the book?\n4. What is the most common word in the book?\n5. What is the most common word in the book which is not a stop-word?\n'

In [11]:
from pyspark import SparkContext

In [138]:
#1
#Read a melville-moby_dick text file and print the first 15 lines.
sc =SparkContext.getOrCreate()
rdd = sc.textFile('melville-moby_dick.txt')


24/09/11 08:23:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [139]:
#2
#How many words are in the book?
rdd_words_num  = rdd.map(lambda x:len(x.split()))
print(rdd_words_num.sum())

212030


In [140]:
#3
#How many unique words are in the book?
word_not_clean = rdd.flatMap(lambda x:x.split())
word_clean = word_not_clean.map(lambda x: (''.join([i for i in x if i.isalpha()]),1))
word_count = word_clean\
    .reduceByKey(lambda a, b: a + b)\
    .sortBy(lambda x:x[1], ascending= False)
word_count.count()

21791

In [141]:
#4
#What is the most common word in the book?
word_count.take(1)

[('the', 13624)]

In [142]:
#5
#What is the most common word in the book which is not a stop-word?
with open('english stop words.txt', 'r', encoding='utf-8') as file:
    content = file.read()
stop_words = content.splitlines()
word_count_wo_sw = word_count.filter(lambda x: x[0] not in stop_words)
word_count_wo_sw.sortBy(lambda x:x[1], ascending= False).take(1)

[('one', 857)]

In [143]:
rdd_stop_words = sc.textFile('english stop words.txt')
rdd_stop_words_t = rdd_stop_words.map(lambda x:(x,1))
word_count.subtractByKey(rdd_stop_words_t).sortBy(lambda x:x[1], ascending= False).take(1)


[('one', 857)]

In [144]:
sc.stop()